# Отток клиентов

Продолжение борьбы с дисбалансом [проекта по прогнозированию оттока клиентов](https://github.com/SanYattsu/Data_Science_Projects/blob/main/Kaggle/Bank_Customer_Churn/Bank-Customer-Churn-Prediction.ipynb) с использованием imblearn.

# Подготовка данных

## Импорт библиотек и загрузка данных

In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import GradientBoostingClassifier

from sklearn.metrics import classification_report

from imblearn.under_sampling import EditedNearestNeighbours
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import make_pipeline

In [2]:
# Предварительная настройка.
pd.options.display.float_format = '{:0.3f}'.format

Загрузим и посмотрим данные.

In [3]:
url = 'https://code.s3.yandex.net/datasets/'

try:
    df = pd.read_csv(url + 'Churn.csv', index_col='RowNumber').reset_index(drop=True)
except:
    print('Не удалось загрузить файл, проверьте путь.')

In [4]:
df.head()

,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,15634602,Hargrave,619,France,Female,42,2.000,0.000,1,1,1,101348.880,1
1,15647311,Hill,608,Spain,Female,41,1.000,83807.860,1,0,1,112542.580,0
2,15619304,Onio,502,France,Female,42,8.000,159660.800,3,1,0,113931.570,1
3,15701354,Boni,699,France,Female,39,1.000,0.000,2,0,0,93826.630,0
4,15737888,Mitchell,850,Spain,Female,43,2.000,125510.820,1,1,1,79084.100,0


Признаки CustomerId и Surname не пригодятся в дальнейшем исследовании, удаляем их.

In [5]:
df.drop(columns=['CustomerId','Surname'], inplace=True)

## Feature engineering и устранение пропусков в данных 

Пропуски есть в столбце Tenure (сколько лет человек является клиентом банка).

Поменяем типы.

In [6]:
df.HasCrCard = df.HasCrCard.astype(bool)
df.IsActiveMember = df.IsActiveMember.astype(bool)
df.Exited = df.Exited.astype(bool)

Создадим новые категории из имеющихся данных.

In [7]:
# По балансу на счете.
df['balance_category'] = pd.cut(df.Balance,
                                bins=[0, 50000, 100000, 150000, np.infty],
                                labels=['low','mid','high','v_high'],
                                include_lowest=True)

In [8]:
# По оценочной заработной плате.
df['salary_category'] = pd.cut(df.EstimatedSalary,
                               bins=[0, 50000, 100000, 150000, np.infty],
                               labels=['low','mid','high','v_high'],
                               include_lowest=True)

In [9]:
# По возрасту.
df['age_category'] = pd.cut(df.Age,
                            bins=[18, 30, 42, 50, np.infty],
                            labels=['t','y','m','o'],
                            include_lowest=True)

Запомним места, в которых встречаются пропуски (Tenure), создав новый признак, и заполним пропуски медианными значениями.

In [10]:
df['is_nan'] = df.Tenure.isna()
df.Tenure = df.Tenure.fillna(df.groupby(['balance_category','salary_category','age_category'])['Tenure'] \
                     .transform('median')) \
                     .map(int)
# Проверим остались ли NaN.
df.Tenure.isna().sum()

0

In [11]:
df.sample()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,balance_category,salary_category,age_category,is_nan
6006,529,France,Female,31,5,0.000,2,True,False,26817.230,False,low,low,y,True


## Построение модели

In [12]:
target = df['Exited']
features = df.drop(columns='Exited')
features = pd.get_dummies(features, drop_first=True)

# Разобьём данные на тренировочную и тестовую выборки.
X_train, X_test, y_train, y_test = train_test_split(
    features, target*1, test_size=0.2, random_state=38
)

print(X_train.shape, y_train.shape)

(8000, 21) (8000,)


In [13]:
pipe = make_pipeline(
    RobustScaler(),
    EditedNearestNeighbours(n_neighbors=2),
    SMOTE(random_state=38),
    GradientBoostingClassifier(random_state=38,
                               learning_rate=0.21,
                               subsample=0.9,
                               n_estimators=85)
)

cross_val_score(pipe, X_train, y_train, cv=5, scoring='f1').mean()

0.6187551675739132

In [14]:
pipe.fit(X_train, y_train)
y_hat = pipe.predict(X_test)
print(classification_report(y_test, y_hat))

              precision    recall  f1-score   support

           0       0.91      0.86      0.88      1588
           1       0.55      0.67      0.61       412

    accuracy                           0.82      2000
   macro avg       0.73      0.77      0.75      2000
weighted avg       0.84      0.82      0.83      2000



# Выводы:

* Метод методы библиотеки imblearn отлично показали себя и могут использоваться для борьбы с несбалансированной выборкой.